# Demo: PostGIS database access via Python

In [1]:
!pip install sshtunnel psycopg2 geopandas

## Simple example: Connect to and query database

In [15]:
import psycopg2
import geopandas as gpd
from sshtunnel import SSHTunnelForwarder

import paramiko
import io
with open('C:/Users/charl/OneDrive/Bureau/PrivateKey3', 'r') as f:
    key = f.read()
    pkey = paramiko.RSAKey.from_private_key(io.StringIO(key))

with SSHTunnelForwarder(('194.163.151.34', 22), ssh_username='flo', remote_bind_address=('172.30.0.4', 5432), ssh_pkey= pkey) as server:
    
    server.start()
    print('Server connected.')

    params = {
        'database': 'eubucco',
        'user': 'NvWkXLveTsmzpuuhXdewHwavJFSsSoVa',
        'password': 'sUur5w7PHUufrRqWvcilJ5Gvxk5TbdAu3cLv9gWXWqd6P1GTTqKqZGgexwtpB6eQ',
        'host': 'localhost',
        'port': server.local_bind_port,
        'connect_timeout': 10,
        }

    conn = psycopg2.connect(**params)
    conn.set_session(readonly=True)
    curs = conn.cursor()
    print('Database connected.')

    sql = 'SELECT * FROM data_country;'
    data = gpd.GeoDataFrame.from_postgis(
        sql=sql,
        con=conn,
        geom_col='geometry', 
        index_col='id', 
        )
    print('Data retrieved.')

data

Server connected.
Database connected.
Data retrieved.


,name,geometry,convex_hull,csv_id,gpkg_id
id,,,,,
30,Italy Other-license,None,None,9432c895-d177-425c-87a7-b6181167f2f8,91ff19bb-11ae-4da3-b246-71296344590b
2,Belgium,"MULTIPOLYGON (((3859958.019 3160527.233, 38601...",0103000020DB0B0000010000001D000000DBA80FDCFE76...,c00bc9a3-3e02-404d-8463-a1e0ce847d59,e0f5c02c-7794-4e2b-91a4-d1150a70ab23
3,Bulgaria,"MULTIPOLYGON (((5766401.396 2319455.017, 57664...",0103000020DB0B00000100000025000000AC6FC021569C...,c69fc2a7-74ec-4488-b39e-e72efa2caeda,708538d9-6666-4971-a459-a2205ed10950
4,Croatia,"MULTIPOLYGON (((4838760.330 2164354.241, 48387...",0103000020DB0B000001000000190000002940B0D19673...,b3f0f151-0318-4c0e-96ef-65ae1bfc47f5,9e5751f2-4a50-4ce6-b3d8-48aec73ab228
5,Cyprus,"MULTIPOLYGON (((6350643.684 1612948.279, 63506...",0103000020DB0B00000100000029000000B7E56A7E8062...,f3824bd6-b1e7-438a-b0d7-55545b88ee8f,81b1eefe-fb98-4efa-a038-e11159995f6e
6,Czechia,"POLYGON ((4662370.809 2843424.355, 4662176.980...",0103000020DB0B000001000000230000006E2E4722E8B2...,5bf0a606-36e6-469b-841b-ea5891767d0c,4d136c05-35af-4fd9-a936-98c8e04001d7
9,Finland,"MULTIPOLYGON (((5021622.613 4131171.471, 50216...",0103000020DB0B0000010000002B0000009E64B14613ED...,0d91b30f-3c97-4a16-8ef5-c45924ab10b9,fc90b8f7-3942-4e69-8ebe-724253d84fcd
10,France,"MULTIPOLYGON (((4258616.576 2027881.387, 42586...",0103000020DB0B0000010000002A00000037E0F3CAFE3E...,6e5ce629-a0b0-4154-830b-646e89f367f2,e9767cbd-0fb7-4b61-9135-d822a7b4061f
8,Estonia,"MULTIPOLYGON (((5106412.320 3926401.969, 51064...",0103000020DB0B0000010000003A0000002150E5DC4C32...,ab4bc0a5-cd00-4dd6-8ccc-2d57baca618b,571ed494-c995-47ff-9abb-6d9c43b08b2b


## Spatial query example: Sending multiple, spatial PostGIS queries

In [18]:
import psycopg2
from sshtunnel import SSHTunnelForwarder
import geopandas as gpd
import pandas as pd
from contextlib import contextmanager

@contextmanager
def db_tunnel():
    with SSHTunnelForwarder(('194.163.151.34', 22), ssh_username='flo', remote_bind_address=('172.30.0.4', 5432)) as server:
        
        server.start()

        params = {
            'database': 'eubucco',
            'user': 'NvWkXLveTsmzpuuhXdewHwavJFSsSoVa',
            'password': 'sUur5w7PHUufrRqWvcilJ5Gvxk5TbdAu3cLv9gWXWqd6P1GTTqKqZGgexwtpB6eQ',
            'host': 'localhost',
            'port': server.local_bind_port,
            'connect_timeout': 10,
            }

        conn = psycopg2.connect(**params)
        conn.set_session(readonly=True)
        
        yield conn


with db_tunnel() as conn:
    results = []
    boundaries = [
        # Paris, 11e Arrondissement
        'POLYGON ((3762754.4419298171997070 2890991.8044788637198508, 3764095.9635624778456986 2888187.3515811231918633, 3761960.7702519670128822 2888983.4717874415218830, 3761945.1074829874560237 2889266.3062656684778631, 3761744.4274468249641359 2890595.5182541240938008, 3762754.4419298171997070 2890991.8044788637198508))',
        # Vichy
        #'POLYGON ((3812062.8628411777317524 2581341.6049629254266620, 3811694.3412095555104315 2581981.0313450326211751, 3812127.4740968858823180 2582555.1165699004195631, 3812626.5119165452197194 2581535.7130569545552135, 3813359.5603073895908892 2581590.2022620658390224, 3813347.0566178392618895 2580818.7682339800521731, 3814345.0114732668735087 2578388.9866586597636342, 3814389.6237756260670722 2577626.1422555106692016, 3813747.5744092036038637 2577943.7211474208161235, 3812019.8191846804693341 2579188.5741550792008638, 3812062.8628411777317524 2581341.6049629254266620))',
    ]
    
    for boundary in boundaries:
        sql = f"""
            select bu.id, bu.id_source, bc.name as country, br.name as region, b.name as city, bu.height, bu.age, bu.type, bu.type_source, bu.geometry
                from data_building as bu
                    RIGHT OUTER JOIN data_country bc on bu.country_id = bc.id
                    RIGHT OUTER JOIN data_region br on br.id = bu.region_id
                    RIGHT OUTER JOIN data_city b on b.id = bu.city_id
                WHERE ST_Within( bu.geometry, 'SRID=3035;{boundary}');
        """

        result = gpd.GeoDataFrame.from_postgis(
            sql=sql,
            con=conn,
            geom_col='geometry', 
            index_col='id', 
            )

        #results.append(result)
        result.to_csv("Paris.csv")

    #data = pd.concat(results)